In [174]:
import pandas as pd
import re
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [175]:
browser = webdriver.Firefox()
browser.get('https://www.tripadvisor.fr')
time.sleep(2)
browser.maximize_window()

# Recherche : restaurants sur Paris

In [176]:
restaurant_btn = browser.find_element(By.CSS_SELECTOR, 'a[data-automation="centralNav_restaurants"]')
restaurant_btn.click()
time.sleep(2)

search_bar = browser.find_element(By.CSS_SELECTOR, 'input[title="Rechercher"]')
search_bar.send_keys('Paris')
time.sleep(3)

browser.find_element(By.ID, 'typeahead_results').find_element(By.TAG_NAME, 'a').click()

# Clique sur un élément

In [177]:
# Fonction pour la page avec tous les restaurants
def get_all_restaurant_divs(browser):
    """Prend en paramètre un WebDriver et renvoie une liste de tous les restaurants (div en HTML).

    Exemple d'utilisation : `get_all_restaurant_divs(browser)`
    """
    elements = browser.find_element(By.CSS_SELECTOR, 'div[class="Ikpld f e"]').find_elements(By.CSS_SELECTOR, 'div[data-test]')
    return [el for el in elements if re.search(r'\d+_list_item', el.get_attribute('data-test'))]

In [178]:
# Fonctions pour la page d'un restaurant

# TODO: modifier pour utiliser une classe ?
def get_restaurant_page_info(browser):
    """Prend en paramètre un WebDriver et récupère le nom, les catégories et l'adresse sur la page d'un restaurant et les renvoie dans un dictionnaire.

    Exemple d'utilisation : `get_restaurant_page_info(browser)`

    Exemple de résultat :
    ```json
    {
      'nom': 'Le Jules Verne',
      'categorie': ['Française', 'Européenne'],
      'adresse': 'Avenue Gustave Eiffel, 75007 Paris France'
    }
    ```
    """
    res_info_div = browser.find_element(By.CSS_SELECTOR, 'div[data-test-target="restaurant-detail-info"]')

    categories = res_info_div.find_elements(By.CLASS_NAME, 'dlMOJ')

    return {
        'nom': res_info_div.find_element(By.TAG_NAME, 'h1').text,
        'categorie': [cat.text for cat in categories if '€' not in cat.text],
        'adresse': res_info_div.find_elements(By.CLASS_NAME, 'AYHFM')[1].text,
        'avis': []
    }

def get_all_review_divs(browser):
    """Prend en paramètre un WebDriver et renvoie une liste de tous les avis (div en HTML).
    
    Exemple d'utilisation : `get_all_review_divs(browser)`
    """
    return browser.find_element(By.CSS_SELECTOR, 'div[data-contextchoice="DETAIL"]').find_elements(By.CSS_SELECTOR, 'div[class="reviewSelector"]')

def get_review_info(review_div):
    """Prend en paramètre un avis (div en HTML) et renvoie un dictionnaire avec sa note, son titre et son texte.
    
    Exemple d'utilisation : 
    ```python
    liste_avis = browser.find_elements(By.CSS_SELECTOR, 'div[class="reviewSelector"]')
    avis = get_avis_info(liste_avis[0])
    ```
    
    Exemple de résultat :
    ```json    
    {
      'note': '50',   
      'titre': 'Très bon restaurant',
      'texte': 'Les plats étaient très bons et le service était excellent.'
    }
    ```
    """
    classe_note = review_div.find_element(By.CLASS_NAME, 'ui_bubble_rating').get_attribute('class').split('_')[-1]

    return {
        'note': int(classe_note) / 10,
        'titre': review_div.find_element(By.CLASS_NAME, 'noQuotes').text,
        'texte': review_div.find_element(By.CLASS_NAME, 'partial_entry').text
    }

In [183]:
def switch_to_next_restaurants_page(browser):
    """Prend en paramètre un WebDriver et clique sur le bouton \"Suivant\" s'il existe.

    Exemple d'utilisation : `switch_to_next_restaurants_page(browser)`
    """
    try:
        next_page_btn = browser.find_element(By.CSS_SELECTOR, 'a[data-smoke-attr="pagination-next-arrow"]')
        
        if next_page_btn:
            browser.execute_script(f'window.scrollTo(0, {next_page_btn.rect["y"] - 300})')

            next_page_btn.click()
    except Exception as e:
        print('Erreur lors de la tentative de changement de page :', e)
        pass

def switch_to_next_reviews_page(browser):
    """Prend en paramètre un WebDriver et clique sur le bouton \"Suivant\" s'il existe.

    Exemple d'utilisation : `switch_to_next_reviews_page(browser)`
    """
    try:
        pagination = browser.find_element(By.CLASS_NAME, 'ui_pagination')
        next_btn = pagination.find_element(By.CLASS_NAME, 'next')

        if next_btn and 'disabled' not in next_btn.get_attribute('class'):
            browser.execute_script(f'window.scrollTo(0, {next_btn.rect["y"] - 300})')

            next_btn.click()
    except Exception as e:
        print('Erreur lors de la tentative de changement de page :', e)
        pass

switch_to_next_reviews_page(browser)

In [180]:
def recup_avis(browser):
    """va dans la boucle while des restaurants"""
    page_avis = 1

    while page_avis < 3:
        print('URL de la page :', browser.current_url)

        # Rafraîchir la liste des avis après chaque changement de page
        avis_html = get_all_review_divs(browser)

        print('Nombre d\'avis trouvés sur cette page :', len(avis_html))

        for avis in avis_html:
            browser.execute_script(f'window.scrollTo(0, {avis.rect["y"] - 300})')
            print('scroll sur avis')
            time.sleep(1)

            # cliquer sur "Plus"
            try:
                plus_btn = avis.find_element(By.CLASS_NAME, 'ulBlueLinks')
                plus_btn.click()
                print('clic sur plus')
                time.sleep(2)

            except:
                pass

            print('checkpoint')
            # Récupération des informations de l'avis
            avis_dict = get_review_info(avis)
            print(avis_dict)
        # continue

In [181]:
index_restaurants = 28
page_restaurants = 1

restaurants = []
avis = []

while page_restaurants < 4:
    print('URL de la page :', browser.current_url)

    # Rafraîchir la liste des restaurants après chaque changement de page
    restaurants_html = get_all_restaurant_divs(browser)

    print('Nombre de restaurants trouvés sur cette page :', len(restaurants_html))

    while index_restaurants < len(restaurants_html):
        try:
            res = restaurants_html[index_restaurants]
            lien_avis = res.find_element(By.TAG_NAME, 'a')

            browser.execute_script(f'window.scrollTo(0, {lien_avis.rect["y"] - 300})')
            time.sleep(2)

            browser.get(lien_avis.get_attribute('href'))
            time.sleep(4)

            # Récupération des informations du restaurant
            res_dict = get_restaurant_page_info(browser)
            print(res_dict)
            restaurants.append(res_dict)


            # TODO: Récupération des avis
            avis_html = get_all_review_divs(browser)
            # page_avis = 1

            recup_avis(browser)


            # Retour à la liste des restaurants
            browser.back()
            time.sleep(2)

            index_restaurants += 1

        # Si la liste de restaurants (div) est obsolète, rafraîchir
        except StaleElementReferenceException:
            restaurants_html = get_all_restaurant_divs(browser)
            continue

    # Si on arrive au bout de la liste des restaurants, on passe à la page suivante
    if index_restaurants >= len(restaurants_html):
        switch_to_next_restaurants_page(browser)
        page_restaurants += 1
        time.sleep(4)
        index_restaurants = 28

print('nombre de restaurants :', len(restaurants))


URL de la page : https://www.tripadvisor.fr/Restaurants-g187147-Paris_Ile_de_France.html


Nombre de restaurants trouvés sur cette page : 30
{'nom': 'Bao & ME', 'categorie': ['Asiatique', 'Fusion', 'Saine'], 'adresse': '89 Rue du Faubourg Poissonnière, 75009 Paris France', 'avis': []}
URL de la page : https://www.tripadvisor.fr/Restaurant_Review-g187147-d14210107-Reviews-Bao_ME-Paris_Ile_de_France.html
Nombre d'avis trouvés sur cette page : 10
scroll sur avis
checkpoint
{'note': 5.0, 'titre': 'Incroyablement bon', 'texte': 'Equipe très agréable Repas absolument délicieux avec beaucoup de saveurs. Rapport qualité prix très intéressant'}
scroll sur avis
checkpoint
{'note': 5.0, 'titre': 'Belle découverte', 'texte': 'Super soirée, accompagnée de super plats et d’une super ambiance. Spéciale mention pour le service très à l’écoute! Une belle découverte avec des plats sans gluten mais aussi pour les végétariens des options possibles.'}
scroll sur avis
checkpoint
{'note': 5.0, 'titre': 'Belle découverte !', 'texte': "Des baos originaux, bons et copieux. L'accueil et le service éta

NoSuchElementException: Message: Unable to locate element: div[class="Ikpld f e"]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16


In [ ]:
test = browser.find_element(By.CSS_SELECTOR, 'span[class="IiChw"]')
test.click()
time.sleep(3)

In [42]:
# Utiliser un sélecteur XPath pour trouver l'élément
elements = browser.find_element(By.CLASS_NAME, 'class="partial_entry"')

print(elements)

NoSuchElementException: Message: Unable to locate element: .rmyCe _G B- z _S c Wc wSSLS jWkoZ XDcOZ; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16
